In [168]:
import sqlite3 as dbms

conn = dbms.connect("students.sqlite3")  # Как оригинально!
cursor = conn.cursor()

In [169]:
with open("11.drop_ddl.sql", 'r', encoding='utf-8') as f:
    drop_ddl = f.read()
with open("11.create_ddl.sql", 'r', encoding='utf-8') as f:
    create_ddl = f.read()

In [170]:
if True:
    cursor.executescript(drop_ddl)
    conn.commit()
if True:
    cursor.executescript(create_ddl)
    conn.commit()

In [171]:
import sqlalchemy
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey
from sqlalchemy.orm import sessionmaker, relationship
from sqlalchemy.ext.declarative import declarative_base

DeclBase = declarative_base()

engine = sqlalchemy.create_engine('sqlite:///students.sqlite3', echo=False)  # echo=True для логгинга
Session = sessionmaker(bind=engine)
session = Session()

In [172]:
class Program(DeclBase):
    __tablename__ = 'programs'
    id = Column(Integer, primary_key=True)
    name = Column(String)

    students = relationship("Student", backref="program")
    programs_courses = relationship("Program_course", backref="program")

    def __init__(self, name):
        self.name = name

class Student(DeclBase):
    __tablename__ = 'students'
    id = Column(Integer, primary_key=True)
    card = Column(String)
    surname = Column(String)
    name = Column(String)
    patronymic = Column(String)
    
    program_id = Column(Integer, ForeignKey('programs.id'))
    marks = relationship("Mark", backref="student")
    
    def __init__(self, card, surname, name, patronymic, program):
        self.card = card
        self.surname = surname
        self.name = name
        self.patronymic = patronymic
        self.program = program
        
class Course(DeclBase):
    __tablename__ = 'courses'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    
    programs_courses = relationship("Program_course", backref="course")
    marks = relationship("Mark", backref="course")
    
    def __init__(self, name):
        self.name = name
        
class Program_course(DeclBase):
    __tablename__ = 'programs_courses'
    semester_number = Column(Integer, primary_key=True)
    
    course_id = Column(Integer, ForeignKey('courses.id'), primary_key=True)
    program_id = Column(Integer, ForeignKey('programs.id'), primary_key=True)
    
    def __init__(self, semester_number, course, program):
        self.semester_number = semester_number
        self.course = course
        self.program = program
        
class Mark(DeclBase):
    __tablename__ = 'marks'
    mark = Column(Integer)
    
    course_id = Column(Integer, ForeignKey('courses.id'), primary_key=True)
    student_id = Column(Integer, ForeignKey('students.id'), primary_key=True)
    
    def __init__(self, mark, student, course):
        self.mark = mark
        self.student = student
        self.course = course

In [173]:
se = Program("Информационные технологии")
se1 = Program("Прикладная математика и физика")

st1 = Student("002002", "Иванов", "Пётр", "Сидорович", se)
st2 = Student("002003", "Петрова", "Исидора", "Ивановна", se)
st3 = Student("002004", "Сидоров", "Александр", "Петрович", se1)
c1 = Course("Алгоритмы и анализ данных")
c2 = Course("Физика твёрдого тела")
pc1 = Program_course("3", c1, se)
pc2 = Program_course("4", c2, se1)
m1 = Mark("5", st1, c1)
m2 = Mark("4", st2, c1)
m3 = Mark("3", st3, c2)


session.add_all([st1, st2, st3, c1, c2, pc1, pc2, m1, m2, m3]) 
session.commit()

In [174]:
import sys

print("Программы и студенты")
for p in session.query(Program):
    print("Программа: ", p.name)
    for s in p.students:
        print("- ", s.name, s.patronymic, s.surname)
    for s in p.programs_courses:
        print("Номер семестра:", s.semester_number)
for p in session.query(Student):
    print("Студент: ", p.name)
    for s in p.marks:
        print("- ", s.mark, "Курс: ", s.course.name)

Программы и студенты
Программа:  Информационные технологии
-  Пётр Сидорович Иванов
-  Исидора Ивановна Петрова
Номер семестра: 3
Программа:  Прикладная математика и физика
-  Александр Петрович Сидоров
Номер семестра: 4
Студент:  Пётр
-  5 Курс:  Алгоритмы и анализ данных
Студент:  Исидора
-  4 Курс:  Алгоритмы и анализ данных
Студент:  Александр
-  3 Курс:  Физика твёрдого тела
